# Feasible Inequality Constraints in Process

When running Process with inequality constraints, it appears that for converged solutions some of the constraints are violated by a small amount. This needs to be fixed for accurate reliability analysis work, where sensitivity to small constraint violations is important. The solution should be truly feasible; i.e. no constraints violated.

## Regular Process run of large-tokamak

Take the large-tokamak regression test input file (`lt_original_IN.DAT`) and run it. The constraint residuals for limit equations are all listed as equality constraints (`lt_original_MFILE.DAT`):

```
Density_upper_limit_______________normalised_residue____________________ (eq_con005)___________________      2.7513E-09    
Neutron_wall_load_upper_limit_____normalised_residue____________________ (eq_con008)___________________      4.2808E-08    
Fusion_power_upper_limit__________normalised_residue____________________ (eq_con009)___________________      5.4311E-08    
```
As all the constraints are treated a equalities, some are negative and some positive.

## Convert to inequality constraints

Comment out f-value iteration variables in input file:
```
9, 14, 26, 21, 103, 46, 25, 36, 35, 38, 39, 50, 51, 53, 54, 106, 110, 113, 123, 154, 117, 48, 49
```
all commented out: f-values removed as optismisation parameters; not allowed to vary (`lt_ineqs_no_f_values`). `neqns` is also specified so Process differentiates between equality and inequality constraints (`lt_ineqs_IN.DAT`).

Solves! All inequality constraints positive (feasible for Process), but one is negative (violated) (`lt_ineqs_MFILE.DAT`):
```
J_winding_pack/J_protection_limit_______________________________________ (ineq_con035)_________________     -1.4590E-07 
```

Process has produced a solution with a violated constraint. This is what we want to avoid.

## Modify VMCON convergence criterion to prevent violated constraints

In VMCON's convergence criterion, added:
```python
if convergence_info < epsilon:
    # All ineqs positive, i.e. feasible
    if np.all(result.ie >= 0.0):
        print("Converged, cons satisfied")
        break
    else:
        print("Converged, but cons violated")
```
On re-running `lt_ineqs_IN.DAT`, the `convergence-info` is below epsilon a few times before finally converging with all inequality constraints being positive (feasible): see `lt_ineqs_mod_vmcon_MFILE.DAT`. The result is a truly feasible solution, with all inequality constraints satisfied.